In [1]:
import os
os.environ["AZURE_OPENAI_API_KEY"] = "84641c2f2c1d46888b9f2e2a1b54cbf2"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://deray-oai.openai.azure.com/"
os.environ['AZURE_OPENAI_API_VERSION'] = '2023-03-15-preview'

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")


In [2]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = AzureChatOpenAI(openai_api_key=AZURE_OPENAI_API_KEY, 
                        model="gpt-3.5-turbo", 
                        azure_deployment="deray-deployment-1", 
                        api_version='2023-03-15-preview',
                        temperature=0.9)

In [3]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [4]:
chain = prompt | model

In [5]:
product = "video indexer and summary app"
chain.invoke(product)

AIMessage(content='"VideoScribe"', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 25, 'total_tokens': 30}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-0f9102b9-7149-4263-b887-50b73b6a65a8-0', usage_metadata={'input_tokens': 25, 'output_tokens': 5, 'total_tokens': 30})

In [12]:
from langchain.chains import SimpleSequentialChain

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = first_prompt | model

In [13]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = second_prompt | model

In [15]:
overall_simple_chain = chain_one | chain_two

In [21]:
product = "bengali music school"

overall_simple_chain.invoke(product)

AIMessage(content='Shonar Bangla Music Academy offers music education and training in traditional Bangladeshi music and dance forms for all skill levels.', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 155, 'total_tokens': 182}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-f51f392a-d490-459f-b28c-b8890e392bfc-0', usage_metadata={'input_tokens': 155, 'output_tokens': 27, 'total_tokens': 182})

In [23]:
from langchain.chains import SequentialChain
from langchain.chains import LLMChain

In [24]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=model, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [25]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=model, prompt=second_prompt, 
                     output_key="summary"
                    )


In [26]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=model, prompt=third_prompt,
                       output_key="language"
                      )


In [27]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=model, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [32]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=False
)

In [33]:
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
 
overall_chain.invoke(review)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate the following review to english:

Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Can you summarize the following review in 1 sentence:

I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: What language is the following review:

Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a follow up response to the following summary in the 

{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds the taste of the product mediocre and suspects that it may be an old or counterfeit batch compared to the ones bought in stores.',
 'followup_message': "Cher critique,\n\nNous sommes désolés d'apprendre que vous avez trouvé le goût du produit médiocre. Nous vous assurons que tous nos produits sont frais et authentiques. Nous prendrons en considération vos préoccupations et nous vous encourageons à nous contacter directement pour discuter de cette question plus en détail.\n\nSincèrement,\n[L'équipe de l'entreprise]"}

In [34]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [35]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [36]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [37]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=model, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [38]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=model, prompt=default_prompt)

In [39]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [40]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(model, router_prompt)

In [41]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [46]:
chain.invoke("Who is Sachin Tendulkar?")



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modification

{'input': 'Who is Sachin Tendulkar?',
 'text': 'Sachin Tendulkar is a former Indian cricketer who is widely regarded as one of the greatest batsmen in the history of the sport. He is known for his exceptional skill, consistency, and longevity in the game, and he holds numerous records in international cricket. Tendulkar is also celebrated for his sportsmanship and humility, and he is considered a national hero in India for his contributions to the game.'}